<a href="https://colab.research.google.com/github/DarekGit/FACES_DNN/blob/master/notebooks/02_00_Miary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### [Spis treści](https://github.com/DarekGit/FACES_DNN/blob/master/notebooks/Praca_Dyplomowa.ipynb)

## 2. Metryki oceny detekcji


W procesie wykrywania i rozpoznawania twarzy kluczowa jest prawidłowa ocena skuteczności detekcji. W poniższych punktach zostały przedstawione podstawowe metryki detekcji. 

W punkcie 2.6. została przedstawiona ujednolicona miara mAP przygotowana do oceny  wszystkich modeli używanych w pracy do detekcji twarzy wraz z przykładem użycia w notebooku: [mAP notebook](../notebooks/02_01_mAP.ipynb "mAP notebook").
<br>Należy zwrócić uwagę, iż w większości repozytoriów i konkursów stosowana jest głównie miara mAP, jednak może ona posiadać nieznaczne modyfikacje w zależności od sposoby implementacji. Implementacja nie powinna mieć istotnego wpływu na wynik, ale nawet różnice poniżej 1pp mogą przesądzać o wyborze modelu do dalszych prac. Dlatego zależało nam na przygotowaniu jednej spójnej miary do oceny wszystkich badanych modeli. 
<br><br>


### 2.1. Intersection Over Union (IOU)

Intersection Over Union jest miarą bazującą na indeksie Jaccarda oceniającego względny stopień dopasowania pól jako stosunek części wspólnej do pola łącznego. Poprzez zastosowanie IoU możemy określić czy mamy przypadek ważnej detekcji (True Positive) czy też nie (False Positive). Domyślnie do detekcji sotosuje się  próg $IoU \ge 50\%$.

$\text{IOU}=\frac{\text{pole}\left(B_{p} \cap B_{gt} \right)}{\text{pole}\left(B_{p} \cup B_{gt} \right)}$

gdzie:<br>
$B_{p}$ oznacza predicted Box (wykryte), $B_{gt}$ oznacza ground truth Box (wskazane)


<br>Obrazek poniżej ilustruje IoU pomiędzy wskazanym polem/obwiednią (w kolorze zielonym - ground truth bounding box) a wykrytym polem/obwiednią (w kolorze czerwonym - detected bounding box.
<br>


<br>

<div align="center">
<img src="https://github.com/DarekGit/Faces_DNN/blob/master/Figures/iou.png?raw=1" alt="Intersection Over Union"  >
<br>
Rys. 1. IoU pomiędzy wskazanym polem (w kolorze zielonym - ground truth bounding box) <br>a wykrytym polem (w kolorze czerwonym - detected bounding box).
</div>
<br>



### 2.2. Precision i Recall

Podstawowe elementy Precision i Recall:

* **True Positive (TP)**: Prawidłowa detekcja. Detekcja z IOU ≥ _threshold_  
* **False Positive (FP)**: Nieprawidłowa detekcja. Detekcja z IOU < _threshold_  
* **False Negative (FN)**: Nie wykryte wskazane pole  (ground truth).  
* **True Negative (TN)**: Reprezentuje prawidłowy brak detekcji obiektu. W przypadku detekcji obiektów (w tym twarzy)  można przypisać bardzo wiele pól oznaczjący prawdziwy brak detekcji obiektu, a które nie są związane z jakością pracy modelu, dlatego nie stosuje się tego elementu w ocenie detektora.

_threshold_: zależy od definicji miary, domyślnie przyjumje się wartośc 0.5.

### Precision

Precyzja określa jaka część wyników uzyskanych w trakcie detekcji jako należących do danej klasy obiektów faktycznie do niej należy. Określa się jako stosunek True Positive do wszystkich uzyskanych detekcji dla danje klasy, podanym wzorem:

<br>
$\text{Precision} = \frac{\text{TP}}{\text{TP}+\text{FP}}=\frac{\text{TP}}{\text{all detections}}$ <br>


### Recall 

Recall (czułość) określa zdolnością modelu do wykrycia wszystkich wskazanych obiektów (ground truth). Jest to stosunek True Positive do wszystkich wskazanych obiektów, podanym wzorem:

<br>
$\text{Recall} = \frac{\text{TP}}{\text{TP}+\text{FN}}=\frac{\text{TP}}{\text{all ground truths}}$ 
<br>
<br>

### 2.3. Krzywa Precision/Recall

Krzywa Precision i Recall jest dobrym sposobem oceny jakości detekcji obiektów, w zależności od stopnia pewności detekcji (confidence), dla danej klasy obiektów. Detektor obiektów uznaje się za dobry jeśli jego Precission pozostaje wysoki dla narastającego Recall, co oznacza, że przy zwiększeniu progu pewności (confidence treshold), Precission and Recall pozostają wysokie.

Zły detektor potrzebuje zwiększyć ilość wykrywanych obiektów (zwiększyć False Positive = mniejsze Precission) w celu wykrycia  wskazanych obiektów (wysokie Recall).
Normalnie krzywa zaczyna się od dużych wartości Precission i maleje wraz ze wzrostem Recall.


### 2.4. Average Precision

Metoda AP została zastosowana w konkursie PASCAL VOC w 2010 roku.
<br><br>


* <b>Interpolacja 11-punktowa</b>
<br>


Dla ułatwienia obliczeń w pierwszych implementacjach AP stosowano uśrednienie krzywej Precision x Recal do jedenastu rownomiernie rozłożonych punktów na osi Recall [0, 0.1, 0.2, ... , 1], wtedy:

<br>
$\text{AP}=\frac{1}{11} \sum_{r\in \left \{ 0, 0.1, ...,1 \right \}}\rho_{\text{interp}\left ( r \right )}$

gdzie:
<br>
$\rho_{\text{interp}} = \max_{\tilde{r}:\tilde{r} \geq r} \rho\left ( \tilde{r} \right )$,<br>
 $\rho\left ( \tilde{r} \right )$ jest Precission dla Recall $\tilde{r}$ .

<br>W tym przypadku, mamy prostą formułę, AP jest obliczane tylko dla interpolacji w 11 punktach $r$ biorąc **maksymalne Precission dla Recall większego od $r$**.
<br>
<br>

* <b>Interpolacja dla wszystkich punktów</b>
<br>

Jest dokładniejszą miarą AP niż interpolacja 11 punktowa, i może być wyrażona następująca formułą:

$\sum_{n=0} \left ( r_{n+1} - r_{n} \right ) \rho_{\text{interp}}\left ( r_{n+1} \right )$
 
<br>gdzie:
<br>$\rho_{\text{interp}}\left ( r_{n+1} \right ) = \max_{\tilde{r}:\tilde{r} \ge r_{n+1}} \rho \left ( \tilde{r} \right )$,<br>
$\rho \left ( \tilde{r} \right )$  jest Precission dla Recall $\tilde{r}$.

<br>W tym przypadku AP obliczane jest dla wszystkich wartości maksymalnych Precission biorąc **maksymalny poziom Precission dla Recall większego lub równego $r+1$**. W ten sposób obliczamy pole pod krzywą interpolowaną (AUC).
<br>

### 2.5. COCO mAP

mAP zostało zastosowane w konkurcie COCO detection, jest bardzo popularną miarą wyrażającą jakość detekcji jedną liczbą dla zestawu IoU. 
<br>Najnowsze artykuły naukowe zwykle podają wyniki również dla COCO.
<br>Z definicji mAP to średnia wartość Average Precision. W przypadku COCO mAP jest liczona poprzez uśrednianie AP dla wybranych IoU we wszystkich klasach obiektów.

Charakterystyczny zapis AP @ [. 5: .95] odpowiada średniemu AP dla IoU od 0.5 do 0.95 z wielkością kroku 0.05. 

W konkursie COCO AP to średnia dla 10 poziomów IoU w 80 kategoriach (AP @ [.50: .05: .95]: zaczynając od 0.5,a kończąc na IoU = 0.95 z krokiem 0.05).

<br>


### 2.6. Moduł mAP.py

Moduł zawiera ujednoliconą miarę mean Average Precision przygotowaną do oceny wszystkich modeli używanych w pracy do detekcji twarzy. 
Moduł zapewnia bardzo łatwe użycie metryki oraz wizualizację wyników.
Metryka bazuje na danych podawanych w postaci list wykrytych  pól oraz pól wskazanych  w opisie datasetu (ground truth).
Koordynaty pól powinny być przedstawione w postaci (left, top, right, bottom], w przypadku nieprawidłowej kolejności przygotowana miara wprowadza odpowiednie poprawki.
<br><BR>

Należy zwrócić uwagę, że w dostępnych materiałach, często pomijana jest kwestia sortowania wyników detekcji. Do określenia prawidłowej miary mAP wyniki detekcji muszą być posortowane malejąco względem pewności detekcji (confidence). <br> I tylko dla takiej kolejności obliczane są kolejne wartości:<br>
<br>$Precission_i = (\frac{\sum_0^i{TP_i}}{\sum_0^i{\left(TP_i+FP_i\right)}})$ <br>oraz <br>$Recall_i = (\frac{\sum_0^i{TP_i}}{All\ Ground\ Truth})$
<br>
<br> W module mAP zasotosowano obliczenie dla wszystkich punktów Recall, krzywe można wyświetlać jako interpolowane lub nie.
<br><br>

Przykład użycia modułu mAP.py podany jest w notebooku:  [02_01_mAP.ipynb](02_01_mAP.ipynb)
<br><br>
Moduł mAP dostępny jest na: https://github.com/DarekGit/mAP

### 2.7. GIoU

W detekcji obiektów najczęściej stosowaną fukcją kosztu jest odległośc L1 lub L2 między koordynatami pól wykrytych a pól wskazanych (ground truth). Prowadzi to do rozbieżności wyników optymalizacji modelu a wartości maksymalnych AP, ponieważ nie ma silnej korelacji pomiędzy IoU a odległością koordynat pól.  Brak korelacji można wykazać na przykładzie dla stałej odległosci L2 pól:
<br>
<br>

<div align="center">

<img src="https://github.com/DarekGit/Faces_DNN/blob/master/Figures/L2 versus IoU.jpg?raw=1" alt="L2 versus IoU" width="500" ><br>


Rys. 2. Dla tej samej odległosci różne wartości IoU.  <a href="https://github.com/DarekGit/FACES_DNN/blob/master/notebooks/Bibliografia.ipynb">[14]</a>
</div>


<br><br> Natomiast zastosowanie IoU w fukcji kosztu jest problematyczne ze względu na brak informacji o odległości pól obiektu przy braku ich przecięcia, IoU=0.
Rozwiązaniem tego zagadnienia jest generalizacja pojęcia IoU przez wpowadzenie elementu minimalnego pola C obejmującego pole wskazane i wykryte przez detektor i zdefinoowanie generalized IoU jako:

<br>$GIoU = IoU - \frac{C - \text{area of Union}}{C}$

<br>Miarę GIoU <a href="https://github.com/DarekGit/FACES_DNN/blob/master/notebooks/Bibliografia.ipynb">[14]</a> stosuje się w optymalizacji funckji kosztu od 2020r jako element uzupełniający do odległości L1 lub L2 przy zastosowaniu odpowiednich wag dla poszczególnych elementów funkcji kosztu.
Minimalizację wykonuje się dla $1 - GIoU$

<br>Przykład użycia GIoU w funkcji kosztu dla modelu DeTr w Detectron2: 
https://github.com/DarekGit/detr/blob/master/util/box_ops.py

<br>


In [ ]:
import torch
from torchvision.ops.boxes import box_area

def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(-1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=-1)


def box_xyxy_to_cxcywh(x):
    x0, y0, x1, y1 = x.unbind(-1)
    b = [(x0 + x1) / 2, (y0 + y1) / 2,
         (x1 - x0), (y1 - y0)]
    return torch.stack(b, dim=-1)


# modified from torchvision to also return the union
def box_iou(boxes1, boxes2):
    area1 = box_area(boxes1)
    area2 = box_area(boxes2)

    lt = torch.max(boxes1[:, None, :2], boxes2[:, :2])  # [N,M,2]
    rb = torch.min(boxes1[:, None, 2:], boxes2[:, 2:])  # [N,M,2]

    wh = (rb - lt).clamp(min=0)  # [N,M,2]
    inter = wh[:, :, 0] * wh[:, :, 1]  # [N,M]

    union = area1[:, None] + area2 - inter

    iou = inter / union
    return iou, union


def generalized_box_iou(boxes1, boxes2):
    """
    Generalized IoU from https://giou.stanford.edu/

    The boxes should be in [x0, y0, x1, y1] format

    Returns a [N, M] pairwise matrix, where N = len(boxes1)
    and M = len(boxes2)
    """
    # degenerate boxes gives inf / nan results
    # so do an early check
    assert (boxes1[:, 2:] >= boxes1[:, :2]).all()
    assert (boxes2[:, 2:] >= boxes2[:, :2]).all()
    iou, union = box_iou(boxes1, boxes2)

    lt = torch.min(boxes1[:, None, :2], boxes2[:, :2])
    rb = torch.max(boxes1[:, None, 2:], boxes2[:, 2:])

    wh = (rb - lt).clamp(min=0)  # [N,M,2]
    area = wh[:, :, 0] * wh[:, :, 1]

    return iou - (area - union) / area

<br>Przykład użycia mAP: 
<b>[mAP notebook](../notebooks/02_01_mAP.ipynb "mAP notebook")

###<br> [3. Bazy danych](03_00_Datasety.ipynb)   
### [Spis treści](https://github.com/DarekGit/FACES_DNN/blob/master/notebooks/Praca_Dyplomowa.ipynb)